# Assignment 3: When will summer Arctic sea ice disappear?
In Part 1 we will fit a linear model to Arctic sea ice extent data. This will be a relatively straightforward task because all you're asked to do is re-use a few lines of code from Assignment 2 and then interpret the results. 

Sources. This part of the assignment uses data from NSIDC: https://nsidc.org/data/nsidc-0051.

In [ ]:
# Load the data
import pickle
import pandas as pd

with open('northern-hemisphere-sea-ice-nsidc.pickle', 'rb') as file:
    sea_ice_data = pickle.load(file)
    sea_ice_df = pd.DataFrame(sea_ice_data)
    sea_ice_df['years'] = 1970+sea_ice_df['date'].astype(int)/1e9/86400/365
    
# Extract just the minima
from scipy.signal import find_peaks
inds = find_peaks(-sea_ice_df['extent'],prominence=1)[0]
min_dates = sea_ice_df.iloc[inds]['years'] # use this variable as the first input to the linear regression
min_extents = sea_ice_df.iloc[inds]['extent'] # use this variable as the second input to the linear regression

# Plot the data
import matplotlib.pyplot as plt

fig,ax=plt.subplots(figsize=(16,9))
plt.plot(sea_ice_df['years'],sea_ice_df['extent'],'.',label='Data')
plt.plot(sea_ice_df.iloc[inds]['years'],sea_ice_df.iloc[inds]['extent'],'o',markersize=10,label='Summertime minima')
plt.plot((1970,2030),(0,0),'-r',label='zero sea ice')
plt.ylim([-1,17.5])
plt.xlim([1978,2020])
plt.ylabel('Sea ice extent, million sq km')
plt.legend()
plt.grid()
plt.show()


### Fitting the linear model.
In this part, you want to fit a linear model between to the time series of minimum extents.  The individual data points consist of pairs of (time, extent) as stored int he variables ```min_dates, min_extents```.

In the cell below, reuse the code from Assignment 2 to fit a straight line to the data.  This will consist of three lines of code that import the linregres function, define the variable ```output```, and then print the output.
#### Write code here:

In [ ]:
from scipy.stats import linregress
output = linregress( min_dates, min_extents )
output

### Interpreting the results.
Once you've correctly carried out the linear regression in the cell above, you'll be left with a ```LinregressResult``` that has properties like ```slope```, ```intercept```, etc.  The linear model describes ```extent = slope*year + intercept```.

### What year does this linear model predict that the Arctic will be, on average, sea ice free?
Solving the above equation, this can be found by evaluating the expression ``` - output.intercept/output.slope```

### The following code calculates confidence intervals. 
You don't need to modify this code, but you do need to extract the upper bound and lower bounds for analysis. Hint: look at the variable names.

In [ ]:
# Calculate upper and lower bounds
import numpy as np
import scipy.stats as stats

model_years = np.linspace(1970, 2100, 100)
expected_model = output.intercept + output.slope * model_years

# Confidence interval
alpha = 0.05 # significance level
tails = 2
n = len(min_dates)
dof = n - 2
t_critical = stats.t.ppf(1 - (alpha / tails), dof)
x = min_dates
ci = 100*t_critical * output.stderr * np.sqrt(1 / n + (model_years - np.mean(x))**2 / np.sum((x - np.mean(x))**2))

upper_bound = expected_model + ci
lower_bound = expected_model - ci

upper_bound_year = round(model_years[np.where(upper_bound<0)].min())
lower_bound_year = round(model_years[np.where(lower_bound<0)].min())

In [ ]:
import numpy as np

plt.subplots(figsize=(16,9))
plt.plot(min_dates, min_extents,'o')

# Line of best fit
plt.plot(model_years, expected_model, 
         label=f'Line of Best Fit, R² = {-output.rvalue:.2f}')

plt.fill_between(
    model_years, upper_bound, lower_bound, facecolor='#b9b9b9', zorder=0,
    label=r'95% Confidence Interval'
)

plt.plot((1970,2200),(0,0),'-')
plt.plot(expected_year,0,'o',label='Expected first year with no sea ice')
plt.plot(lower_bound_year,0,'o',label='95% Confidence lower bound')
plt.plot(upper_bound_year,0,'o',label='95% Confidence upper bound')
plt.ylim([-1,10])
plt.xlim([1978,2100])
plt.legend()
plt.show()